# Flappy Bird

Dzisiaj zrobimy własnego klona Flappy Birda. Będzie to pewien krok do przodu, ponieważ 1) wykorzystamy animacje, 2) grę przeznaczymy z góry na telefon. Oryginał Flappy Birda jest frustrująco trudny do grania, więc będziemy też mieli możliwość obniżenia poziomu trudności.

## Proces tworzenia - Paczka narzędzi

Zaczniemy od ściągnięcia z Unity Asset Store paczki z narzędziami do stworzenia Flappy Birda. Paczka oferuje nawet gotową grę, ale my się zadowolimy zestawem tekstur i fontów.

![](./data/flappy-assets.png)

## Proces tworzenia - Flappy

Naszym głównym elementem będzie Flappy, dziwaczny ptaszek. Jego podstawowymi elementami będą `Rigidbody2D`, `CapsuleCollider2D` oraz `FlappyScript`, gdzie ten ostatni będzie wklejonym przez nas skryptem:

```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class FlappyScript : MonoBehaviour
{

    public float FlyHeight = 1;
    public float Cooldown = 0.05f;
    public float HorizontalSpeed = 1;
    public float rotation = 0;

    private float timer = 0;

    public bool KillMePlz = false;
    
    public bool Started = false;

    // Start is called before the first frame update
    void Start()
    {
        var physics = GetComponent<Rigidbody2D>();
        physics.bodyType = RigidbodyType2D.Static;
    }

    // Update is called once per frame
    void Update()
    {
        var physics = GetComponent<Rigidbody2D>();
        if (!Started && Input.GetButtonUp("Jump"))
        {
            physics.bodyType = RigidbodyType2D.Dynamic;
            physics.velocity = Vector2.right*HorizontalSpeed;
            Started = true;
            return;
        }


        if (KillMePlz) return;

        timer += Time.deltaTime;
        if (Input.GetButtonUp("Jump") && timer > Cooldown)
        {
            timer = 0;
            var velocity = physics.velocity;
            velocity.y = 0;
            physics.velocity = velocity;

            var FlyForce = Mathf.Sqrt(FlyHeight) / physics.gravityScale;
            physics.AddForce(Vector2.up*FlyForce, ForceMode2D.Impulse);
        }

        rotation = Mathf.Lerp(rotation, physics.velocity.y * 10, 0.1f);

        transform.rotation = Quaternion.AngleAxis(rotation, Vector3.forward);

    }

    private void OnCollisionEnter2D(Collision2D collision)
    {
        KillMePlz = true;
        var physics = GetComponent<Rigidbody2D>();
        physics.bodyType = RigidbodyType2D.Kinematic;
        physics.velocity = Vector2.down;
    }
}

```

## Proces tworzenia - Kamera

Dodamy również skrypt do kamery który zapewni nam śledzenie ptaka przez kamerę. Do kamery dodamy skrypt `CameraScript`:

```csharp
using System.Collections;
using System.Collections.Generic;
using UnityEngine;

public class CameraScript : MonoBehaviour
{

    public GameObject Bird;

    // Start is called before the first frame update
    void Start()
    {
        
    }

    // Update is called once per frame
    void Update()
    {
        var CameraPos = transform.position;
        var BirdPos = Bird.transform.position.x;
        if (BirdPos > CameraPos.x)
        {
            CameraPos.x = BirdPos;
            transform.position = CameraPos;
        }
    }

}
```

Po uzupełnieniu skryptu będziemy również musieli wskazać obiekt który ma śledzić kamera, w tym wypadku naszego ptaśka.

## Proces tworzenia - Game Manager

Ostatnim elementem będzie nasz nieśmiertelny Game Manager - pusty obiekt trzymajacy skrypt odpowiedzialny za wyświetlanie punktów i generowanie nowych kolumn. W przypadku śmierci Flappy'ego - a będzie się to powtarzało cały czas - Manager ma również załadować jeszcze raz bieżący poziom. Tworzymy komponent `GameManagerScript`:

```csharp
using System.Collections.Generic;
using UnityEngine;
using UnityEngine.SceneManagement;
using UnityEngine.UI;

public class GameManagerScript : MonoBehaviour
{
    public GameObject ColumnObject;
    public float ColumnsDelta = 2;


    public List<GameObject> ColumnsList = new List<GameObject>();

    public float CurrentGoal;
    public GameObject Flappy;

    private float LastColumnPos;

    public float MaxColumnDistance = 20;
    public float MaximalBreakHeight = 3;
    public float MaximalColumnBreak = 3;

    public float MinimalBreakHeight = -1;

    public float MinimalColumnBreak = 1;
    public int Points;

    private float timer;


    // Start is called before the first frame update
    private void AddColumns(float x, float y, float distance)
    {
        ColumnsList.Add(Instantiate(ColumnObject, new Vector2(x, y - distance / 2), Quaternion.identity));
        ColumnsList.Add(Instantiate(ColumnObject, new Vector2(x, y + distance / 2),
            Quaternion.AngleAxis(180, Vector3.forward)));
        LastColumnPos = x;
    }

    private void Start()
    {
        AddColumns(2, 0, 2);
        CurrentGoal = 2;
    }

    public float DeathWaitTime = 2;

    // Update is called once per frame
    private void Update()
    {
        var flappyPos = Flappy.transform.position.x;
        while (flappyPos + MaxColumnDistance > LastColumnPos)
        {
            var height = Random.Range(MinimalBreakHeight, MaximalBreakHeight);
            var delta = Random.Range(MinimalColumnBreak, MaximalColumnBreak);
            AddColumns(LastColumnPos + ColumnsDelta, height, delta);
        }

        if (flappyPos > CurrentGoal)
        {
            Points++;
            CurrentGoal += ColumnsDelta;
            var text = GameObject.Find("Text");
            if (text) text.GetComponent<Text>().text = Points.ToString();
        }

        if (Flappy.GetComponent<FlappyScript>().KillMePlz)
        {
            timer += Time.deltaTime;
            if (timer > DeathWaitTime)
            {
                var text = GameObject.Find("Text");
                if (text)
                {
                    text.GetComponent<Text>().text = "PRESS BUTTON TO CONTINUE";
                    text.GetComponent<Text>().fontSize = 40;
                }

                if (Input.GetButtonUp("Jump")) SceneManager.LoadScene(0);
            }
        }
    }
}
```